This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-observe-tutorials/edge-observability-assays).

##  In-Line Model Updates at the Edge Tutorial: Preparation

Models published through Wallaroo to an OCI Registry, then deployed on edge devices, are able to perform in-line updates of the model.  These include:

* Updating a pipeline model step and replace the published pipeline version with the new one:  In this example, a model step is replace by a new model in a pipeline with an edge location already assigned to the pipeline.  The inference logs and other data remains part of the pipeline, though the input and output schemas may change depending on the model.
* Create a new pipeline with a new publish, and replace another pipeline's edge location publish.  The location is transferred to the new pipeline, while the inference logs for the edge location are now added to the new pipeline.  The old inference results are stored in the original pipeline.

* **IMPORTANT NOTE**:  Assays and other log data is based on the pipeline and model name.  If these values change, assays that are currently running will no longer generate analyses.  These should be paused, and new assays based on the current pipeline and model name should be created.  For more information, see [Model Drift Detection with Model Insights](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-observe/wallaroo-pipeline-assays/). 

## Goal

* Package a model in Wallaroo and perform sample inferences.  Publish the pipeline and create an edge location, then deploy the model to an edge location.
* Perform sample inferences on the edge location and verify the inference logs are added to the pipeline associated with the edge location.
* Create a new pipeline version with a new model step and perform sample inferences.  Update the publish with the new pipeline version, and verify that the model is deployed to the edge location deployment.
* Perform sample inferences on the edge location to verify the new model is being used, and verify the inference logs are added to the pipeline associated with the edge location.
* Create a new pipeline with a different model and perform sample inferences.  Publish the pipeline, then replace the publish used for the edge location.  Verify that the edge location is transferred to the new pipeline.
* Perform sample inferences on the edge location to verify model is updated from the new pipeline is being used, and verify the inference logs are added to the new pipeline associated with the edge location.

### Resources

This tutorial provides the following:

* Models:
  * `models/rf_model.onnx`: The champion model that that predict house price prices.
  * `models/xgb_model.onnx`: A new challenger model that takes the same inputs as `models/rf_model.onnx` and outputs house prices with a field of the same name.
  * `models/gbr_model.onnx`: A new challenger model that takes the same inputs as `models/rf_model.onnx` and outputs house prices with a field of the same name.

### Prerequisites

* A deployed Wallaroo instance with [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) and [Edge Observability enabled](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#set-edge-observability-service).
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
* A X64 Docker deployment to deploy the model on an edge location.


## Steps

* Deploying a sample ML model used to determine house prices based on a set of input parameters.
* Publish the model deployment configuration to an OCI registry.
* Use the publish and set edge locations.
* Deploy the model to two different edge locations.

### Import Libraries

The first step will be to import our libraries, and set variables used through this tutorial.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import json

workspace_name = f'in-line-edge-replacement-tutorial'

# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

The following helper function is used to create a new workspace, or retrieve an existing one.

In [2]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace.

Workspace names are unique across the Wallaroo instance.  If a user requests a workspace they have not been granted access to, an error message is returned.

In [4]:
workspace = get_workspace(workspace_name, wl)

wl.set_current_workspace(workspace)

{'name': 'in-line-edge-replacement-tutorial', 'id': 7, 'archived': False, 'created_by': '526c72b4-2a0b-4ade-8f96-8d6dfa642a82', 'created_at': '2024-03-26T17:18:30.459718+00:00', 'models': [{'name': 'rf-house-price-estimator', 'versions': 2, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 3, 26, 18, 0, 41, 147787, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 3, 26, 17, 18, 31, 771534, tzinfo=tzutc())}, {'name': 'xgb-house-price-estimator', 'versions': 4, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 3, 26, 18, 26, 44, 936066, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 3, 26, 17, 34, 18, 853479, tzinfo=tzutc())}, {'name': 'gbr-house-price-estimator', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 3, 26, 18, 31, 59, 387596, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 3, 26, 18, 31, 59, 387596, tzinfo=tzutc())}], 'pipelines': [{'name': 'in-line-edge-replacement', 'create_time': datetime.datetime(20

### Upload The Champion Model

For our example, we will upload the champion model that has been trained to derive house prices from a variety of inputs.  The model file is `rf_model.onnx`, and is uploaded with the name `rf-house-price-estimator`.

In [5]:
housing_model_control = (wl.upload_model("rf-house-price-estimator", 
                                        './models/rf_model.onnx', 
                                        framework=Framework.ONNX)
                                        .configure(tensor_fields=["tensor"])
                        )

### Build the Pipeline

This pipeline is made to be an example of an existing situation where a model is deployed and being used for inferences in a production environment.  We'll call it `edge-inline-replacement-demo`, set `housing_model_control` as a pipeline step, then run a sample inference.

In [6]:
mainpipeline = wl.build_pipeline('edge-inline-replacement-demo')


# clear the steps if used before
mainpipeline.clear()

mainpipeline.add_model_step(housing_model_control)

#minimum deployment config
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()

mainpipeline.deploy(deployment_config = deploy_config)

name,edge-inline-replacement-demo
created,2024-03-26 18:53:24.630184+00:00
last_updated,2024-03-26 18:53:25.142374+00:00
deployed,True
arch,None
accel,None
tags,
versions,"5a1bf0a2-9595-41de-9852-44d211c2c0fb, a5105a19-7215-4e6b-9a8a-5ddcc6aa875c"
steps,rf-house-price-estimator
published,False


### Testing

We'll use a single row inference request that should return a result of around `$700k`.

In [8]:
normal_input = pd.DataFrame.from_records({"tensor": [[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]]})
result = mainpipeline.infer(normal_input)
display(result)

,time,in.tensor,out.variable,anomaly.count
0,2024-03-26 18:53:44.662,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.7],0


### Undeploy Main Pipeline

With the sample inferences complete, we will undeploy the main pipeline and return the resources back to the Wallaroo instance.

In [9]:
mainpipeline.undeploy()

name,edge-inline-replacement-demo
created,2024-03-26 18:53:24.630184+00:00
last_updated,2024-03-26 18:53:25.142374+00:00
deployed,False
arch,None
accel,None
tags,
versions,"5a1bf0a2-9595-41de-9852-44d211c2c0fb, a5105a19-7215-4e6b-9a8a-5ddcc6aa875c"
steps,rf-house-price-estimator
published,False


## Edge Deployment

We can now deploy the pipeline to an edge device.  This will require the following steps:

* Publish the pipeline:  Publishes the pipeline to the OCI registry.
* Add Edge:  Add the edge location to the pipeline publish.
* Deploy Edge:  Deploy the edge device with the edge location settings.

### Publish Pipeline

Publishing the pipeline uses the pipeline `wallaroo.pipeline.publish()` command.  This requires that the Wallaroo Ops instance have [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) enabled.

The following publishes the pipeline to the OCI registry and displays the container details.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Edge Publication](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/).

In [10]:
pub = mainpipeline.publish()

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing...... Published.


### Add Edge Location

The edge location is added with the [`wallaroo.pipeline_publish.add_edge(name)`](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-reference-guide/pipeline_publish/#PipelinePublish.add_edge) method.  This returns the OCI registration information, and the `EDGE_BUNDLE` information.  The `EDGE_BUNDLE` data is a base64 encoded set of parameters for the pipeline that the edge device is associated with, the workspace, and other data.

For full details, see [Wallaroo SDK Essentials Guide: Pipeline Edge Publication: Edge Observability](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/#edge-observability).

For this example, we will add two locations:

* `houseprice-edge-inline-replacement-demo`

These will be used in later steps for demonstrating inferences through different locations.

In [11]:
edge_name_01 = "houseprice-edge-inline-replacement-demo"
edge_publish_01 = pub.add_edge(edge_name_01)
display(edge_publish_01)

ID,10
Pipeline Name,edge-inline-replacement-demo
Pipeline Version,90643d74-8883-451c-aa2b-4c41d9af04f8
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4781
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-inline-replacement-demo:90643d74-8883-451c-aa2b-4c41d9af04f8
Helm Chart URL,oci://ghcr.io/wallaroolabs/doc-samples/charts/edge-inline-replacement-demo
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:d930bc56895eaf32785fe6e907a01cd4aa5b6d78f6d60caf056ff7e96dcba880
Helm Chart Version,0.0.1-90643d74-8883-451c-aa2b-4c41d9af04f8
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 4.0, 'memory': '3Gi'}, 'requests': {'cpu': 4.0, 'memory': '3Gi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': None}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 0.2, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}}"
User Images,[]


### DevOps Deployment

For the model deployment to the edge location, we will use the `Helm Install Command` from the previous step.  The following variables must be set first.

* `$REGISTRYURL`: The URL of the OCI registry service hosting the pipeline publish.
* `$OCI_USERNAME`: The username for the OCI registry service.
* `$OCI_PASSWORD`: The password or token for the OCI registry service.
* `$HELM_INSTALL_NAME`: The name of the Helm based installation for the edge deployment.
* `$HELM_INSTALL_NAMESPACE`: The name of the namespace to set the edge based deployment to.

Before deploying, verify `helm` has access to the target registry.  Using the variable above, this command is:

```bash
helm registry login $REGISTRYURL --username $OCI_USERNAME --password $OCI_PASSWORD
```

If the command succeeds, proceed.  If there are any issues, verify the settings above before continuing.

Once the helm registry service is set, deploy using the `Helm Install Command` from `edge_publish_01`.

Once deployed, use the following to port forward to the `helm` based deployment.

```bash
kubectl port-forward svc/engine-svc -n $HELM_INSTALL_NAMESPACE 8080 --address 0.0.0.0
```

For more details on model edge deployments with Wallaroo, see [Model Operations: Run Anywhere](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/).

### Edge Inference Examples

With the model deployed on the edge location, we will perform sample inferences on the edge deployment.  For this example, the hostname is `testboy.local`.  Adjust the hostname to fit your edge location deployment.

The following endpoints are available.

| Endpoint | Type | Description |
|---|---|---|
| `/models` | GET | Returns the models and model versions used in the edge deployment. |
| `/pipelines` | GET | Returns the pipeline ID and status of the pipeline used in the edge deployment. |
| `/infer` | POST | The inference request endpoint.  This accepts either an Apache Arrow table, or a JSON in pandas record format.

For more details on model edge deployments with Wallaroo, see [Model Operations: Run Anywhere](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/).

First we'll retrieve the model.  We should see the same name we set when we uploaded it to the Wallaroo instance.

In [16]:
!curl testboy.local:8080/models

{"models":[{"model_version":{"name":"rf-house-price-estimator","visibility":"private","workspace_id":7,"conversion":{"python_version":"3.8","requirements":[],"framework":"onnx"},"id":9,"image_path":null,"status":"ready","task_id":null,"file_info":{"version":"1c23e843-a173-4890-87a3-2ae5f9fe1518","sha":"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6","file_name":"rf_model.onnx"},"created_on_version":"2024.1.0"},"config":{"id":18,"model_version_id":9,"runtime":"onnx","filter_threshold":null,"tensor_fields":["tensor"],"input_schema":null,"output_schema":null,"batch_config":null,"sidekick_uri":null},"status":"Running"}]}

Now we'll retrieve the pipeline details, and verify the name is the same as set earlier and it is running.

In [18]:
!curl testboy.local:8080/pipelines

{"pipelines":[{"id":"edge-inline-replacement-demo","status":"Running"}]}

Now do the infer with the same DataFrame we used for our sample inference when the model was deployed in the Wallaroo instance.

In [17]:
import json

df = pd.DataFrame.from_records({"tensor": [[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]]})

data = df.to_dict(orient="records")

!curl -X POST testboy.local:8080/infer \
    -H "Content-Type: Content-Type: application/json; format=pandas-records" \
    --data '{json.dumps(data)}'

[{"time":1711479575506,"in":{"tensor":[4.0,2.5,2900.0,5505.0,2.0,0.0,0.0,3.0,8.0,2900.0,0.0,47.6063,-122.02,2970.0,5251.0,12.0,0.0,0.0]},"out":{"variable":[718013.7]},"anomaly":{"count":0},"metadata":{"last_model":"{\"model_name\":\"rf-house-price-estimator\",\"model_sha\":\"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6\"}","pipeline_version":"90643d74-8883-451c-aa2b-4c41d9af04f8","elapsed":[351790,773467],"dropped":[],"partition":"houseprice-edge-inline-replacement-demo"}}]

### Retrieve Inference Logs

The edge deployment location is tied to the pipeline, so when we check the pipeline logs, we will see in the `metadata.partition` the location where the inference was performed.  From the list, we should see the same name as set to our location.

In [19]:
display(mainpipeline.logs(dataset=['time', 'out', 'metadata']).loc[:, ['time', 'out.variable', 'metadata.last_model', "metadata.partition"]])

,time,out.variable,metadata.last_model,metadata.partition
0,2024-03-26 18:59:35.506,[718013.7],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",houseprice-edge-inline-replacement-demo
1,2024-03-26 18:53:44.662,[718013.7],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",engine-858996678c-qvd2w


### Change Pipeline Steps

For the next demonstration, we will use the same pipeline and update the pipeline steps with a new model.  This time with the model `./models/xgb_model.onnx`, which takes the same input as `./models/rf_model.onnx`.  We will also set a different name for this model to distinguish it from the one previously uploaded and used as the pipeline step.

Once set, we will deploy the pipeline with the new model, and perform a new inference request.  From this, we will see that this new model outputs a slightly different prediction that the previous one - closer to `$650k`.

In [20]:
housing_model_challenger01 = (wl.upload_model("xgb-house-price-estimator", 
                                        './models/xgb_model.onnx', 
                                        framework=Framework.ONNX)
                                        .configure(tensor_fields=["tensor"])
                        )

mainpipeline.clear()

mainpipeline.add_model_step(housing_model_challenger01)

#minimum deployment config
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()

mainpipeline.deploy(deployment_config = deploy_config)

name,edge-inline-replacement-demo
created,2024-03-26 18:53:24.630184+00:00
last_updated,2024-03-26 19:00:27.973497+00:00
deployed,True
arch,None
accel,None
tags,
versions,"85058098-d395-4af4-98ce-25754175547f, 90643d74-8883-451c-aa2b-4c41d9af04f8, 5a1bf0a2-9595-41de-9852-44d211c2c0fb, a5105a19-7215-4e6b-9a8a-5ddcc6aa875c"
steps,rf-house-price-estimator
published,True


In [21]:
normal_input = pd.DataFrame.from_records({"tensor": [[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]]})
result = mainpipeline.infer(normal_input)
display(result)

,time,in.tensor,out.variable,anomaly.count
0,2024-03-26 19:07:40.093,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[659806.0],0


With this demonstration complete, we undeploy the pipeline to return the resources back to the cluster.

In [29]:
mainpipeline.undeploy()

name,edge-inline-replacement-demo
created,2024-03-26 18:53:24.630184+00:00
last_updated,2024-03-26 19:07:42.040041+00:00
deployed,False
arch,None
accel,None
tags,
versions,"446aeed9-2d52-47ae-9e5c-f2a05ef0d4d6, 85058098-d395-4af4-98ce-25754175547f, 90643d74-8883-451c-aa2b-4c41d9af04f8, 5a1bf0a2-9595-41de-9852-44d211c2c0fb, a5105a19-7215-4e6b-9a8a-5ddcc6aa875c"
steps,rf-house-price-estimator
published,True


#### Replace Pipe Publish with new Pipeline Version

From our pipeline, we will publish the new version.  The method `wallaroo.pipeline.Pipeline.publish()` creates a new publish from the most recent pipeline version and takes the following parameters.

| Parameter | Type | Description |
|---|---|---|
| **deployment_config** | *wallaroo.deployment_config.DeploymentConfig* (*Optional*) | The deployment configuration used for the edge deployment.  By default, this is the same deployment configuration used for the pipeline. |
| **replaces** | *[List[wallaroo.pipeline_publish]]* (*Optional*) | The pipeline publish(es) to replace.|

When a pipeline published is replaced with a new one, the edge locations are transferred to the pipeline that the publish came from.  In this example, this is the same pipeline.  Inference results from the edge location deployments are stored with the pipeline that generated the publish.  Note that if the model or pipeline steps have changed from one pipeline version to the next, the pipeline log schema will change with it.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Log Management](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-logs/).

We will replace our publish earlier, labeled `pub`, with the new publish generated from the pipeline labeled `new_pub`.

In [22]:
new_pub = mainpipeline.publish(replaces=[pub])
new_pub

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing...... Published.


ID,11
Pipeline Name,edge-inline-replacement-demo
Pipeline Version,446aeed9-2d52-47ae-9e5c-f2a05ef0d4d6
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4781
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-inline-replacement-demo:446aeed9-2d52-47ae-9e5c-f2a05ef0d4d6
Helm Chart URL,oci://ghcr.io/wallaroolabs/doc-samples/charts/edge-inline-replacement-demo
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:268a8b7b22b3ab9a62127f56ac5152d264fec194212d2100550ecb6dd6b1cc37
Helm Chart Version,0.0.1-446aeed9-2d52-47ae-9e5c-f2a05ef0d4d6
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 4.0, 'memory': '3Gi'}, 'requests': {'cpu': 4.0, 'memory': '3Gi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': None}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 0.2, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}}"
User Images,[]


Note that in the `Replaces` section, updates are given for each edge location.  We'll continue with the `helm` version, this time with `helm upgrade` to make a new revision of the edge deployment.  The pipeline steps and models will be changed at the model edge location.

Before we do so, we will verify the edge locations associated for our pipeline.  From our example, we will have just the one as created earlier.

In [23]:
mainpipeline.list_edges()

ID,Name,Tags,SPIFFE ID
4a9f77c4-c9bc-4e4b-b59e-b232b80ebce7,houseprice-edge-inline-replacement-demo,[],wallaroo.ai/ns/deployments/edge/4a9f77c4-c9bc-4e4b-b59e-b232b80ebce7


#### Inference Requests with New Pipeline Version

Once the `helm update` procedure is complete, we will verify the models and pipeline as before.  We will see the new model name and model version used.

In [25]:
!curl testboy.local:8080/models

{"models":[{"model_version":{"name":"xgb-house-price-estimator","visibility":"private","workspace_id":7,"conversion":{"python_version":"3.8","requirements":[],"framework":"onnx"},"id":10,"image_path":null,"status":"ready","task_id":null,"file_info":{"version":"8e9ad556-1ea0-4e87-b592-5c0d74bee16a","sha":"31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c","file_name":"xgb_model.onnx"},"created_on_version":"2024.1.0"},"config":{"id":20,"model_version_id":10,"runtime":"onnx","filter_threshold":null,"tensor_fields":["tensor"],"input_schema":null,"output_schema":null,"batch_config":null,"sidekick_uri":null},"status":"Running"}]}

In [26]:
!curl testboy.local:8080/pipelines

{"pipelines":[{"id":"edge-inline-replacement-demo","status":"Running"}]}

We now perform the same inference request on the edge location deployment, and show the results match that as the pipeline with the new model.

In [27]:
import json

df = pd.DataFrame.from_records({"tensor": [[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]]})

data = df.to_dict(orient="records")

!curl -X POST testboy.local:8080/infer \
    -H "Content-Type: Content-Type: application/json; format=pandas-records" \
    --data '{json.dumps(data)}' > inferenceoutput.df.json

# get the dataframe from what we just did

df_result = pd.read_json('./inferenceoutput.df.json', orient="records")
df_result.loc[:, ['time', 'out', 'metadata']]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   627  100   499  100   128  17357   4452 --:--:-- --:--:-- --:--:-- 22392


,time,out,metadata
0,1711480296422,{'variable': [659806.0]},"{'last_model': '{""model_name"":""xgb-house-price-estimator"",""model_sha"":""31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c""}', 'pipeline_version': '446aeed9-2d52-47ae-9e5c-f2a05ef0d4d6', 'elapsed': [221664, 638204], 'dropped': [], 'partition': 'houseprice-edge-inline-replacement-demo'}"


We then show the pipeline logs to demonstrate the inference request at the edge location is added to the pipeline logs.

In [28]:
display(mainpipeline.logs(dataset=['time', 'out', 'metadata']).loc[:, ['time', 'out.variable', 'metadata.last_model', "metadata.partition"]])

,time,out.variable,metadata.last_model,metadata.partition
0,2024-03-26 19:11:36.422,[659806.0],"{""model_name"":""xgb-house-price-estimator"",""model_sha"":""31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c""}",houseprice-edge-inline-replacement-demo
1,2024-03-26 18:59:35.506,[718013.7],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",houseprice-edge-inline-replacement-demo
2,2024-03-26 18:53:44.662,[718013.7],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",engine-858996678c-qvd2w
3,2024-03-26 19:07:40.093,[659806.0],"{""model_name"":""xgb-house-price-estimator"",""model_sha"":""31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c""}",engine-66d588899-qwcfx


### Replace Edge Location from New Pipeline

For this set of examples, we will upload the third model, `./models/gbr_model.onnx` and create a new pipeline named `new-edge-inline-replacement`.  This model is added as a pipeline step and deployed.

In [44]:
housing_model_challenger02 = (wl.upload_model("gbr-house-price-estimator", 
                                        './models/gbr_model.onnx', 
                                        framework=Framework.ONNX)
                                        .configure(tensor_fields=["tensor"])
                        )

new_pipeline = wl.build_pipeline("new-edge-inline-replacement")
# clear the steps if used before
new_pipeline.clear()

new_pipeline.add_model_step(housing_model_challenger02)

#minimum deployment config
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()

new_pipeline.deploy(deployment_config = deploy_config)

name,new-edge-inline-replacement
created,2024-03-26 19:15:46.571741+00:00
last_updated,2024-03-26 19:15:47.072911+00:00
deployed,True
arch,None
accel,None
tags,
versions,"964ac37f-4bf0-430b-bdd1-a0f4a8f0b012, b70c6f36-58d5-4d94-b939-f5af09fba1c5"
steps,gbr-house-price-estimator
published,False


We now perform an inference request on the deployed pipeline, and show the results which should be around `$700k`.

In [45]:
normal_input = pd.DataFrame.from_records({"tensor": [[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]]})
result = new_pipeline.infer(normal_input)
display(result)

,time,in.tensor,out.variable,anomaly.count
0,2024-03-26 19:16:01.986,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[704901.9],0


With the example complete, we undeploy the pipeline to return resources back to the cluster.

In [46]:
new_pipeline.undeploy()

name,new-edge-inline-replacement
created,2024-03-26 19:15:46.571741+00:00
last_updated,2024-03-26 19:15:47.072911+00:00
deployed,False
arch,None
accel,None
tags,
versions,"964ac37f-4bf0-430b-bdd1-a0f4a8f0b012, b70c6f36-58d5-4d94-b939-f5af09fba1c5"
steps,gbr-house-price-estimator
published,False


#### Publish the New Pipeline and Replace Location

We will now publish the new pipeline, and replace the previous edge location publish - labeled `new_pub` with this new one - labeled `new_pipeline_pub`.

We will a new update to the edge location, with a new `helm upgrade` command to use.


In [47]:
new_pipeline_pub = new_pipeline.publish(replaces=[new_pub])
new_pipeline_pub

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing...... Published.


ID,13
Pipeline Name,new-edge-inline-replacement
Pipeline Version,00921571-3db7-4bf9-94dc-377aab558475
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4781
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/new-edge-inline-replacement:00921571-3db7-4bf9-94dc-377aab558475
Helm Chart URL,oci://ghcr.io/wallaroolabs/doc-samples/charts/new-edge-inline-replacement
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:fe9a2d04f09723d5f09d0bc83d4b02367c91bdfc442007a9881e64b2f6eccfad
Helm Chart Version,0.0.1-00921571-3db7-4bf9-94dc-377aab558475
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 4.0, 'memory': '3Gi'}, 'requests': {'cpu': 4.0, 'memory': '3Gi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': None}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 0.2, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}}"
User Images,[]


With the edge location replacement complete with the new pipeline, we list the edges to the original pipeline and the new one to show the edge location is transferred to the new pipeline.

In [48]:
mainpipeline.list_edges()

[]

In [49]:
new_pipeline.list_edges()

ID,Name,Tags,SPIFFE ID
4a9f77c4-c9bc-4e4b-b59e-b232b80ebce7,houseprice-edge-inline-replacement-demo,[],wallaroo.ai/ns/deployments/edge/4a9f77c4-c9bc-4e4b-b59e-b232b80ebce7


#### Sample Inference Requests with New Pipeline Edge Location

Once we have performed the `helm upgrade` command, we will run through the `/models` and `/pipelines` endpoints, then perform an inference request.  We will see the name of the new model - and the new pipeline used for the same edge location.

In [52]:
!curl testboy.local:8080/models

{"models":[{"model_version":{"name":"gbr-house-price-estimator","visibility":"private","workspace_id":7,"conversion":{"python_version":"3.8","requirements":[],"framework":"onnx"},"id":12,"image_path":null,"status":"ready","task_id":null,"file_info":{"version":"759aa90b-b0c4-4b50-87f8-37bda4d69a58","sha":"ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a","file_name":"gbr_model.onnx"},"created_on_version":"2024.1.0"},"config":{"id":24,"model_version_id":12,"runtime":"onnx","filter_threshold":null,"tensor_fields":["tensor"],"input_schema":null,"output_schema":null,"batch_config":null,"sidekick_uri":null},"status":"Running"}]}

In [53]:
!curl testboy.local:8080/pipelines

{"pipelines":[{"id":"new-edge-inline-replacement","status":"Running"}]}

In [54]:
# new inference

import json

df = pd.DataFrame.from_records({"tensor": [[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]]})

data = df.to_dict(orient="records")

!curl -X POST testboy.local:8080/infer \
    -H "Content-Type: Content-Type: application/json; format=pandas-records" \
    --data '{json.dumps(data)}' > inferenceoutput.df.json

# get the dataframe from what we just did

df_result = pd.read_json('./inferenceoutput.df.json', orient="records")
df_result.loc[:, ['time', 'out', 'metadata']]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   627  100   499  100   128  18816   4826 --:--:-- --:--:-- --:--:-- 24115


,time,out,metadata
0,1711480680645,{'variable': [704901.9]},"{'last_model': '{""model_name"":""gbr-house-price-estimator"",""model_sha"":""ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a""}', 'pipeline_version': '00921571-3db7-4bf9-94dc-377aab558475', 'elapsed': [296090, 747861], 'dropped': [], 'partition': 'houseprice-edge-inline-replacement-demo'}"


With the inference request complete, we can check the logs of our new pipeline and verify the edge location inference requests are uploaded to it.

In [55]:
display(new_pipeline.logs(dataset=['time', 'out', 'metadata']).loc[:, ['time', 'out.variable', 'metadata.last_model', "metadata.partition"]])

,time,out.variable,metadata.last_model,metadata.partition
0,2024-03-26 19:18:00.645,[704901.9],"{""model_name"":""gbr-house-price-estimator"",""model_sha"":""ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a""}",houseprice-edge-inline-replacement-demo
1,2024-03-26 19:16:01.986,[704901.9],"{""model_name"":""gbr-house-price-estimator"",""model_sha"":""ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a""}",engine-5c5c799fbf-scxjs
